In [2]:
import os
from datetime import datetime
from typing import Callable, Dict, Literal, Optional, Union

from typing_extensions import Annotated

from autogen import (
    Agent,
    AssistantAgent,
    ConversableAgent,
    GroupChat,
    GroupChatManager,
    UserProxyAgent,
    config_list_from_json,
    register_function,
)
from autogen.agentchat.contrib import agent_builder
from autogen.coding import DockerCommandLineCodeExecutor, LocalCommandLineCodeExecutor

config_list = [
    {"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]},
]

In [31]:
task = (
    "Solve this problem and explain your solution: There exists a unique increasing geometric sequence of five 2-digit positive integers. What is their sum?"
)
print(task)

Solve this problem and explain your solution: There exists a unique increasing geometric sequence of five 2-digit positive integers. What is their sum?


In [32]:
admin = UserProxyAgent(
    name="Admin",
    system_message="Give the task and call on the next agent. After each agent's response, assess the given information and choose with agent is called next to contribute to solving the problem. Instruct the next agent to perform their task. Don't solve the problem.",
    description="Admin. You provide the task and decide which agent speaks next. You manage the conversation in the group chat.",
    code_execution_config=False,
    human_input_mode="NEVER",
    llm_config={"config_list": config_list},
    is_termination_msg=lambda msg: "terminate" in msg.get("content","").lower(),
)

planner = AssistantAgent(
    name="Planner",
    system_message="Given a task, please determine the best way to solve the math problem. List the steps out. Do not solve the problem.",
    description="Planner. You list the steps needed in order to solve the math problem. You do not solve the problem, you simply list the steps.",
    llm_config={"config_list": config_list},
)

calculator = AssistantAgent(
    name="Calculator",
    llm_config={"config_list": config_list},
    system_message="Please solve the math problem based on the steps given by the planner.",
    description="Calculator. You solve the math problem using the steps provided by the planner. ",
)

checker = AssistantAgent(
    name="Checker",
    llm_config={"config_list": config_list},
    system_message="Please check the solution to make sure it was solved correctly using the right steps. If there is an error, identify the error and call the calculator in order to fix the error and get the right solution. Do not solve the problem; only state the errors. If the answer is correct, return 'TERMINATE'",
    description="Checker. You check the solution to make sure it's correct. If it's not, you prompt the needed corrections.",
)

groupchat = GroupChat(
    agents=[admin, planner, calculator, checker],
    messages=[],
    allowed_or_disallowed_speaker_transitions={
        admin: [planner, calculator, checker],
        planner: [admin, calculator],
        calculator: [admin, checker],
        checker: [admin, calculator],
    },
    speaker_transitions_type="allowed",
)

manager = GroupChatManager(groupchat=groupchat, llm_config={"config_list": config_list})

chat_history = admin.initiate_chat(
    manager,
    message=task,
)

Admin (to chat_manager):

Solve this problem and explain your solution: There exists a unique increasing geometric sequence of five 2-digit positive integers. What is their sum?

--------------------------------------------------------------------------------

Next speaker: Planner

Planner (to chat_manager):

Here are the steps to solve this problem:

1. Based on the problem statement, you have an increasing geometric sequence of five 2-digit positive integers.  

2. First, you need to understand a geometric sequence exists when every term after the first is found by multiplying the previous term by a fixed, non-zero number called the "common ratio".

3. Let's denote the first term of the sequence as "a" and the common ratio as "r". As the sequence is increasing, r must be greater than 1. 

4. The five terms can be written as: a, a*r, a*r^2, a*r^3, a*r^4. 

5. Given that the terms are 2-digit integers, and it's an increasing sequence, we can infer that "a" needs to be somewhere betwee